In [1]:
# Se importan las bibliotecas 
import pandas as pd 
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Se carga la base de datos
df=pd.read_csv("recursos_humanos.csv")
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
# EL primer paso es remover la variable que no se pide analizar que es "number_project"
df.drop(["number_project"],axis=1, inplace=True )
df.head()

,satisfaction_level,last_evaluation,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,157,3,0,1,0,sales,low
1,0.80,0.86,262,6,0,1,0,sales,medium
2,0.11,0.88,272,4,0,1,0,sales,medium
3,0.72,0.87,223,5,0,1,0,sales,low
4,0.37,0.52,159,3,0,1,0,sales,low


In [4]:
# Cambiremos los valores del salario a valores numericos
#  0= low(bajo), 1= medium (medio), 2= high(alto) 
df["salary"] = df["salary"].replace({"low": 0, "medium": 1, "high": 2})
df.head()

,satisfaction_level,last_evaluation,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,157,3,0,1,0,sales,0
1,0.80,0.86,262,6,0,1,0,sales,1
2,0.11,0.88,272,4,0,1,0,sales,1
3,0.72,0.87,223,5,0,1,0,sales,0
4,0.37,0.52,159,3,0,1,0,sales,0


In [5]:
# En este ejercicio no se pide un deparyamento en especifico, por lo primero buscaremos sacar todos los deparmentos
tipos_trabajo=[]
for s in df["sales"]:
    if not s in tipos_trabajo:
        tipos_trabajo.append(s)
tipos_trabajo

['sales',
 'accounting',
 'hr',
 'technical',
 'support',
 'management',
 'IT',
 'product_mng',
 'marketing',
 'RandD']

In [6]:
# Una vez realizado esto tammbien cambiaremos  por valores númericos como lo hicimos con salarios 
#0=Sales 1=accouting 2=hr 3=technical 4=support 5=management 6=IT 7=product management 8=marketing 9=RandD

df["sales"] = df["sales"].replace({"sales": 0, "accounting": 1, "hr": 2, "technical":3, "support":4, "management":5, "IT":6,
                                   "product_mng":7,  "marketing":8,"RandD":9} )
df.head()

,satisfaction_level,last_evaluation,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,157,3,0,1,0,0,0
1,0.80,0.86,262,6,0,1,0,0,1
2,0.11,0.88,272,4,0,1,0,0,1
3,0.72,0.87,223,5,0,1,0,0,0
4,0.37,0.52,159,3,0,1,0,0,0


In [7]:
# Definimos la "x" y "y" con las que trabajaremos 
y=np.asarray(df["left"])
x=np.asarray(df.drop(["left"], axis=1))
x[0:1]

array([[  0.38,   0.53, 157.  ,   3.  ,   0.  ,   0.  ,   0.  ,   0.  ]])

In [8]:
#Creación de grupos de entrenamiento y prueba 
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=2)

# Modelación con Radial Basis Funtion (RBF)

In [10]:
#Cargamos la biblioteca correspondiente 
from sklearn import svm
clf=svm.SVC(kernel="rbf")
clf.fit(x_train,y_train)

#Predición para la base de prueba 
yhat=clf.predict(x_test)

In [11]:
#Creación de matriz de confusión
from sklearn.metrics import confusion_matrix
y_pred=clf.predict(x)
cm=confusion_matrix(y_test,yhat)
cm

array([[3394,    0],
       [1042,   64]], dtype=int64)

In [12]:
# Estadísticas de desempeño
from sklearn.metrics import classification_report 
cnf_matrix=confusion_matrix(y_test, yhat, labels=[0,1])
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87      3394
           1       1.00      0.06      0.11      1106

    accuracy                           0.77      4500
   macro avg       0.88      0.53      0.49      4500
weighted avg       0.82      0.77      0.68      4500



Como vemos en la respuesta de del 1 Que es donde el empleado renuncia tiene un recall y f1 score muy bajo por lo cual la predicción basada en este método no es confiable. Ya que solo se pronosticaron 64 empleados en la posición [1,1] como que iban a dejar, pero en realidad tuvimos una cantidad de falsos negativos de 1042. Se pronostican mucho más empleados que van a renunciar a los que si lo hacen.

# Prueba de SVM con Kernel Lineal

In [15]:
clf=svm.SVC(kernel="linear")
clf.fit(x_train,y_train)

yhat=clf.predict(x_test)

#confusión matrix
from sklearn.metrics import confusion_matrix
y_pred=clf.predict(x)
cm=confusion_matrix(y_test,yhat)
cm

array([[3201,  193],
       [ 809,  297]], dtype=int64)

In [16]:
cnf_matrix=confusion_matrix(y_test, yhat, labels=[0,1])
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86      3394
           1       0.61      0.27      0.37      1106

    accuracy                           0.78      4500
   macro avg       0.70      0.61      0.62      4500
weighted avg       0.75      0.78      0.74      4500



Si bien este metodo aumenta el nivel de prediccion de los empleado que renuncia, baja el nivel de los empleado que deciden quedarse, por la precencia de falsos negativos que eran nulos la vez pasada. Por lo cual tampoco podemos utilizar este metodo de manera confiable.

# Prueba de SVM con Kernel polinomial 

In [19]:
clf=svm.SVC(kernel="poly")
clf.fit(x_train,y_train)

yhat=clf.predict(x_test)

#confusión matrix
from sklearn.metrics import confusion_matrix
y_pred=clf.predict(x)
cm=confusion_matrix(y_test,yhat)
cm

array([[3394,    0],
       [1106,    0]], dtype=int64)

In [20]:
cnf_matrix=confusion_matrix(y_test, yhat, labels=[0,1])
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86      3394
           1       0.00      0.00      0.00      1106

    accuracy                           0.75      4500
   macro avg       0.38      0.50      0.43      4500
weighted avg       0.57      0.75      0.65      4500



Este es el método con perores resultados, ya que no predijo que ninguna persona renunciara a la compañía. Es poco confiable.

# Prueba de SVM con Kernel Sigmoide

In [23]:
clf=svm.SVC(kernel="sigmoid")
clf.fit(x_train,y_train)

yhat=clf.predict(x_test)

#confusión matrix
from sklearn.metrics import confusion_matrix
y_pred=clf.predict(x)
cm=confusion_matrix(y_test,yhat)
cm

array([[2732,  662],
       [ 687,  419]], dtype=int64)

In [24]:
cnf_matrix=confusion_matrix(y_test, yhat, labels=[0,1])
print(classification_report(y_test,yhat))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      3394
           1       0.39      0.38      0.38      1106

    accuracy                           0.70      4500
   macro avg       0.59      0.59      0.59      4500
weighted avg       0.70      0.70      0.70      4500



Si bien es otro método que da mejores resultados en cuanto a las personas que renuncian esto lo hace perdiendo de nuevo confiabilidad en las personas que se quedan en el trabajo.

# Conclusión Final

Creo que los metodos fueron poco efectivos por que se tomaron gente de todos los departamentos. Mi sugerencia seria volver a realizar los análisis pero por departamento para ver si eso da mejores resultados que nos ayuden a tomar mejores decisiones para la empresa.